In [174]:
import pandas as pd
import numpy as np
import re
#pd.set_option('display.max_rows',None)
# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

# Question 1

In [175]:
Energy = pd.read_excel('assets/Energy Indicators.xls', header = None, usecols='C:F', skiprows = 18, skipfooter = 38)
cols = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
Energy.columns = cols
Energy.set_index('Country', inplace=True)
ren  = {"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"}


In [176]:
def conversion(item):
    if item == '...':
        return None
    else:
        new_val =  item*1000000
        return new_val

In [177]:
Energy['Energy Supply'] = Energy['Energy Supply'].apply(conversion)

In [178]:
def cleaning(item):

    result = re.sub('\d', '', item) #remove digits
    result = re.sub(r'\([^)]*\)', '', result) #Remove things in parenthesis
    result = result.rstrip()
    
    return result

Energy.index = Energy.index.map(cleaning)
Energy.rename(index = ren, inplace = True)


In [179]:
GDP = pd.read_csv('assets/world_bank.csv', skiprows=4, index_col= 0)

In [180]:
rena = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
dropcol =  list(range(2006,2016))
dropcol = map(str, dropcol)  
#GDP.drop(GDP.columns.difference(dropcol), 1, inplace= True)
GDP = GDP[dropcol]
GDP.index.rename('Country', inplace = True)


In [181]:
GDP.rename(index = rena, inplace = True)
print(GDP.shape)
GDP.head()


(264, 10)


,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,
Aruba,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
Andorra,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
Afghanistan,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
Angola,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
Albania,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


In [182]:
ScimEn = pd.read_csv('assets/scimagojr-3.csv')
ScimEn.head()


,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


In [183]:
ScimEn_short = ScimEn[ScimEn.index < 15]
new_df = ScimEn_short.merge(GDP, how = 'inner', left_on = 'Country', right_on = 'Country')
new_df.set_index('Country')
new_df = new_df.merge(Energy, how = 'inner', left_on='Country', right_on = 'Country')
new_df.set_index('Country', inplace = True)


In [184]:
sortby = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations','Citations per document', 'H index', 'Energy Supply','Energy Supply per Capita', '% Renewable','2006', '2007', '2008','2009', '2010', '2011', '2012', '2013','2014', '2015']
new_df = new_df[sortby]


In [185]:
entries = list()
entries.append(ScimEn.shape[0])
entries.append(GDP.shape[0])
entries.append(Energy.shape[0])
entries

[191, 264, 227]

# Part 3

In [186]:
years = list(range(2006,2016))
years = list(map(str,years))


In [188]:
new_df['avg'] = new_df[years].mean(axis = 1)
new_df.sort_values('avg', axis = 0,ascending = False, inplace= True)
gdp_serie = new_df['avg']


Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avg, dtype: float64